In [2]:
import pandas as pd
import json
import os

In [3]:
meta = pd.read_csv('/home/siddhesh/Documents/Visual Question Answering/images/metadata/images.csv')
meta.head()

,image_id,height,width,path
0,010-mllS7JL,106,106,14/14fe8812.jpg
1,01dkn0Gyx0L,122,122,da/daab0cad.jpg
2,01sUPg0387L,111,111,d2/d2daaae9.jpg
3,1168jc-5r1L,186,186,3a/3a4e88e6.jpg
4,11RUV5Fs65L,30,500,d9/d91ab9cf.jpg


In [31]:
import os
import json
import pandas as pd

json_folder = 'listings/metadata'  # Replace with your JSON folder path
csv_file = 'images/metadata/images.csv'    # Replace with your CSV file path
output_csv = 'merged_metadata_final.csv'

# Load image CSV
csv_df = pd.read_csv(csv_file)

# Prepare a list to hold JSON data
json_records = []

# Loop through each JSON file in the folder
for filename in os.listdir(json_folder):
    if filename.endswith(".json"):
        with open(os.path.join(json_folder, filename), 'r', encoding='utf-8') as f:
            for line in f:
                try:
                    data = json.loads(line)
                    data_flat = {}

                    # Add main_image_id for joining
                    data_flat["main_image_id"] = data.get("main_image_id")

                    # Extract required fields only
                    for field in ["bullet_point", "item_name"]:
                        val = data.get(field)

                        # If it's a list of dictionaries, filter by "en*" language_tag
                        if isinstance(val, list):
                            filtered = [
                                v.get("value") for v in val
                                if isinstance(v, dict)
                                and v.get("language_tag", "").startswith("en")
                                and "value" in v
                            ]
                            data_flat[field] = "; ".join(filtered) if filtered else None

                        elif isinstance(val, dict):
                            # Filter if language_tag starts with "en"
                            if val.get("language_tag", "").startswith("en"):
                                data_flat[field] = val.get("value")
                            else:
                                data_flat[field] = None
                        else:
                            data_flat[field] = val

                    json_records.append(data_flat)

                except json.JSONDecodeError as e:
                    print(f"Skipping line in {filename}: JSONDecodeError - {e}")

# Create a DataFrame from JSON records
json_df = pd.DataFrame(json_records)

# Merge with image CSV on image_id
merged_df = pd.merge(csv_df, json_df, left_on="image_id", right_on="main_image_id", how="inner")

# Keep only required columns
columns_to_keep = ["image_id", "height", "width", "path", "bullet_point", "item_name"]
merged_df = merged_df[columns_to_keep]

# Drop rows with nulls in required columns
merged_df = merged_df.dropna(subset=columns_to_keep)

# Drop rows where bullet_point is empty or just whitespace
merged_df = merged_df[merged_df['bullet_point'].apply(lambda x: isinstance(x, str) and x.strip() != '')]

# Save to CSV
merged_df.to_csv(output_csv, index=False)
print(f"✅ Cleaned & merged CSV written to: {output_csv}")


✅ Cleaned & merged CSV written to: merged_metadata_final.csv


In [28]:
merged_data = pd.read_csv('merged_metadata.csv')
merged_data.head()

,image_id,height,width,path,bullet_point,item_name
0,11xjmNF5TAL,117,88,ee/ee239f0f.jpg,Roomy playing tower for cats: L*W*H =48.5*48.5...,UMI by Amazon Modern Wood Cat Tree Cats Multi ...
1,211PkHPVD6L,500,207,14/1434e124.jpg,Medidas: 196 x 51 x 45 cm; Diseño moderno y br...,Marca Amazon - Movian Moselle - Aparador con v...
2,21480aGcowL,158,498,55/55d6744e.jpg,RESISTENTE A LA CORROSIÓN: esta barra para toa...,Eono Essentials - Toallero de barra autoadhesi...
3,215GfWP2YBL,165,236,ce/ce90c508.jpg,Juego de toallas lujoso de 6 unidades con 2 to...,"Pinzon Juego de toallas de algodón egipcio , 6..."
4,21MMsRD3K3L,500,500,1c/1cab1979.jpg,"Stoff - 100% seide, Hochwertigste Klasse-6A La...",UMI Seide Kissenbezug 80x80cm aus 100% Maulbee...


In [32]:
len(merged_data)

131225

In [33]:
merged_data['item_name'][101]

'UMI Seide Kissenbezug 50x75cm aus 100% Maulbeerseide 25 Momme Kissenhülle für Haar und Haut Marineblau, Reißverschluss, Doppelseitiger Seide'